In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
train_csv_path = "../input/cassava-leaf-disease-classification/train.csv"
label_json_path = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
images_dir_path = "../input/cassava-leaf-disease-classification/train_images"

In [3]:
train_csv = pd.read_csv(train_csv_path)
train_csv['label'] = train_csv['label'].astype('string')

label_class = pd.read_json(label_json_path, orient='index')
label_class = label_class.values.flatten().tolist()
print("Label names :")
for i, label in enumerate(label_class):
    print(f" {i}. {label}")

Label names :
 0. Cassava Bacterial Blight (CBB)
 1. Cassava Brown Streak Disease (CBSD)
 2. Cassava Green Mottle (CGM)
 3. Cassava Mosaic Disease (CMD)
 4. Healthy


In [4]:
train_csv.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [5]:
BATCH_SIZE = 32
IMG_SIZE = 100

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2
)

valid_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split = 0.2
)

In [7]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_csv,
    directory = images_dir_path,
    x_col = "image_id",
    y_col = "label",
    target_size = (IMG_SIZE, IMG_SIZE),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    shuffle = True,
    subset = 'training'
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe = train_csv,
    directory = images_dir_path,
    x_col = "image_id",
    y_col = "label",
    target_size = (IMG_SIZE,IMG_SIZE),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    shuffle = False,
    subset = 'validation'
)

Found 17118 validated image filenames belonging to 5 classes.
Found 4279 validated image filenames belonging to 5 classes.


In [8]:
model = Sequential()
model.add(Conv2D(64,(5,5),input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(128,(5,5),padding = 'same',strides = (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
#model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))

# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# model.add(Dense(64))
# model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(5, activation='softmax'))

In [9]:
model.compile(loss = 'binary_crossentropy',
             optimizer='adam',
             metrics = ['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 64)        4864      
_________________________________________________________________
activation (Activation)      (None, 96, 96, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 256)         2

In [11]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, 
                           patience=3, mode='min', verbose=1)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,'chkpt_{epoch}')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                         save_weights_only=True)

In [12]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,callbacks = [early_stop,checkpoint_callback])

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
534/534 [==============================] - 287s 532ms/step - loss: 0.4378 - accuracy: 0.5880 - val_loss: 0.3527 - val_accuracy: 0.6257
Epoch 2/100
534/534 [==============================] - 195s 364ms/step - loss: 0.3659 - accuracy: 0.6255 - val_loss: 0.3415 - val_accuracy: 0.6290
Epoch 3/100
534/534 [==============================] - 193s 360ms/step - loss: 0.3537 - accuracy: 0.6314 - val_loss: 0.3370 - val_accuracy: 0.6328
Epoch 4/100
534/534 [==============================] - 194s 362ms/step - loss: 0.3412 - accuracy: 0.6351 - val_loss: 0.3603 - val_accuracy: 0.6318
Epoch 5/100
534/534 [==============================] - 253s 473ms/step - loss: 0.3352 - accuracy: 0.6458 - val_loss: 0.3207 - val_accuracy: 0.6436
Epoch 6/100
534/534 [==============================] - 194s 364ms/step - loss: 0.3362 - accuracy: 0.6351 - val_loss: 0.3318 - val_accuracy: 0.6360
Epoch 7/100
534/534 [==============================] - 195s 364ms/step - loss: 0.3275 - accuracy: 0.6425 - val_loss: 0

In [13]:
preds = []
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model.predict(img/255)
    preds.append(np.argmax(prediction))

my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
my_submission.to_csv('submission.csv', index=False) 

In [14]:
print("Submission File: \n---------------\n")
print(my_submission.head())

Submission File: 
---------------

         image_id  label
0  2216849948.jpg      3
